# **Project 2**

**Deadline:** December 01, 2024, 11:59pm ET.



**Group Members:**

Dorothy (Gracie) Rehberg - drehberg1@student.gsu.edu

Lilly Parham - lparham2@student.gsu.edu

Pamela Alvarado-Zarate - palvaradozarate1@student.gsu.edu



**Team Name:** Eco-Warriors




# **Comments / Brainstorming 11/13**
### **EV Registration Data by County/Zip**: https://www.atlasevhub.com/materials/state-ev-registration-data/

# Import Libraries:

When working with data analysis, downloading files, and geospatial data, it's essential to have the right tools at our disposal. Each of the libraries we've imported serves a specific purpose, making it easier to handle various aspects of our project.

The following libraries will allow us to:
- Efficiently download and extract data from various sources.
- Manipulate and analyze structured datasets using pandas.
- Visualize data to extract insights using matplotlib and seaborn.
- Handle geospatial data with geopandas for mapping and spatial analysis.
- Perform geocoding and template-based string manipulations to automate repetitive tasks.

In [ ]:
# Importing necessary libraries for data retrieval, extraction, and analysis:

# Libraries for downloading and handling files:
import os # os: for joining filenames and filepaths
import io # io: for handling input and output operations, particularly in-memory file handling
import zipfile # zipfile: for extracting compressed files

# Libraries for making HTTP requests and retrieving data:
import requests # requests: for sending HTTP requests to access online data

# Data manipulation, analysis, and visualization:
import numpy as np # numpy: for matrix operations
import pandas as pd # pandas: for data manipulation and analysis, especially for working with structured data
import matplotlib.pyplot as plt # matplotlib.pyplot: for creating plots, charts, and visualizations
import seaborn as sns # seaborn: for statistical data visualization, enhancing matplotlib's capabilities

# Geographic libraries:
from shapely.geometry import Point
import geopandas as gpd # geopandas: for reading shapefiles and performing geometric calculations

# libraries for candidate models:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

# libraries for model tuning:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from string import Template
google_drive_file_download_url = Template("https://drive.usercontent.google.com/uc?id=$file_id&authuser=0&export=download")

# Load datasets:
Loading the datasets into the code is a critical first step in any data analysis, exploration, and machine learning project. This will allow us to explore, clean, and transform your data, which in turn sets the stage for building and evaluating machine learning models.

Below we have linked and imported all the neccesary files needed to address our business problem and determine a business solution.

In [ ]:
# Function to download and load state abbreviations data from the US Census.
def load_postal_codes():
    """
    State abbreviations reference from the US Census
    https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html

    Filename:
    state.txt

    URL for direct download:
    https://www2.census.gov/geo/docs/reference/state.txt
    """
    # Google Drive file ID for accessing the state abbreviations data
    file_id = "1L0_KbZAdouKTDt5wX1A_g1CxHWQ1dToj"
    # Construct the file download URL using the provided Google Drive file ID
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    # download the data from url and read into a pandas DataFrame
    return pd.read_csv(
        file_url,
        delimiter="|", # this file is pipe-separated, rather than comma-separated
        dtype = {'STATE': 'string'}, # need to make sure the FIPS code gets read as a string so it doesn't cut off leading zero
    )

def load_FIPS_codes():
    """
    FIPS code reference from the US Census for county and county equivalents
    https://www.census.gov/library/reference/code-lists/ansi.html#cou

    Filename:
    national_county2020.txt

    URL for direct download:
    https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt
    """
    file_id = "15OPIhf823UYcrlWipZ-uUxNYxdH9bW4o"
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    df = pd.read_csv(
        file_url,
        delimiter="|", # this file is pipe-separated, rather than comma-separated
        dtype = { # need to make sure the FIPS code gets read as a string so it doesn't cut off leading zero
            'STATE': 'string',
            'STATEFP': 'string',
            'COUNTYFP': 'string',
            'COUNTYNS': 'string',
            'CLASSFP': 'string',
            'FUNCSTAT': 'string',
        } )
    df['FIPS'] = df['STATEFP'] + df['COUNTYFP']
    return df

# Load the state postal codes into a DataFrame
state_df = load_postal_codes()
# Create a dictionary mapping state names to state abbreviations
states_to_abbreviations = state_df.set_index('STATE_NAME').STUSAB
# Create a dictionary mapping FIPS codes to state abbreviations
FIPS_to_abbreviations = state_df.set_index('STATE').STUSAB
# Display the first few rows of the DataFrame for verification
print(state_df.head())

FIPS_codes_df = load_FIPS_codes()
Name_to_FIPS = FIPS_codes_df.set_index(["STATE", "COUNTYNAME"]).FIPS


  STATE STUSAB  STATE_NAME  STATENS
0    01     AL     Alabama  1779775
1    02     AK      Alaska  1785533
2    04     AZ     Arizona  1779777
3    05     AR    Arkansas    68085
4    06     CA  California  1779778


In [ ]:
def load_ZipCode_FIPS_conversion():
    """
    Table to convert from Zip postal codes to County FIPS codes
    from the Department of Housing and Urban Development
    Cannot be downloaded from a stable url (requires active session with cookies)

    https://www.huduser.gov/portal/datasets/usps_crosswalk.html

    Filename:
    ZIP_COUNTY_092024.xlsx
    """
    file_id = '1BReI2u0Hg_2vwMMpTCOhk5Fh03tORdi_'
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    df = pd.read_excel(file_url)
    for col in ["ZIP", "COUNTY"]:
        df[col] = df[col].astype(str).str.zfill(5)
    return df

ZipCode_FIPS_df = load_ZipCode_FIPS_conversion()
print(ZipCode_FIPS_df)

         ZIP COUNTY USPS_ZIP_PREF_CITY USPS_ZIP_PREF_STATE  RES_RATIO  \
0      00501  36103         HOLTSVILLE                  NY   0.000000   
1      00601  72001           ADJUNTAS                  PR   0.997449   
2      00601  72081           ADJUNTAS                  PR   0.002551   
3      00602  72003             AGUADA                  PR   0.999413   
4      00602  72117             AGUADA                  PR   0.000587   
...      ...    ...                ...                 ...        ...   
54559  99925  02198            KLAWOCK                  AK   1.000000   
54560  99926  02198         METLAKATLA                  AK   1.000000   
54561  99927  02198        POINT BAKER                  AK   0.000000   
54562  99928  02130          WARD COVE                  AK   0.000000   
54563  99929  02275           WRANGELL                  AK   1.000000   

       BUS_RATIO  OTH_RATIO  TOT_RATIO  
0       1.000000   0.000000   1.000000  
1       0.994975   0.987805   0.997051  


In [ ]:
# Function to download and load US state population estimates data from the Census Bureau.
def load_population_estimates():
    """
    From the Census State Population by Characteristics: 2020-2023
    Page: https://www.census.gov/data/datasets/time-series/demo/popest/2020s-state-detail.html
    Datasets: https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/state/asrh/
    Codebooks: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/
    Our specific codebook: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/SC-EST2023-AGESEX-CIV.pdf

    Filename:
    sc-est2023-agesex-civ.csv

    URL for direct download:
    https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/state/asrh/sc-est2023-agesex-civ.csv
    """
    # Google Drive file ID to access the specific population estimates dataset
    file_id = '1azekJW1FKwhdjXuqhn7bTLhGMJ-qfzgW'
    # Generate the download URL using the Google Drive file ID
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    # Read the CSV data into a pandas DataFrame
    df = pd.read_csv(file_url)
    # Select only the relevant columns from the dataset
    df = df[["STATE", "NAME", "SEX", "AGE", "POPEST2023_CIV"]]
    # Remove the row that corresponds to "United States" as a whole to focus on state-specific data
    df = df[df["NAME"] != "United States"]
    # Map full state names to their corresponding abbreviations using the 'states_to_abbreviations' dictionary
    df["State"] = df["NAME"].apply(states_to_abbreviations.get)
    return df

# Load the population estimates data and store it in a DataFrame, printing the head of the data for verification
population_estimates_df = load_population_estimates()
print(population_estimates_df.head())


     STATE     NAME  SEX  AGE  POPEST2023_CIV State
261      1  Alabama    0    0           57885    AL
262      1  Alabama    0    1           58419    AL
263      1  Alabama    0    2           58006    AL
264      1  Alabama    0    3           59016    AL
265      1  Alabama    0    4           59779    AL


In [ ]:
# Function to download and load US state population estimates data from the Census Bureau.
def load_population_estimates_county():
    """
    From the Census County Population by Characteristics: 2020-2023
    Page: https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-detail.html
    Datasets: https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/counties/asrh/
    Codebooks: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/
    Our specific codebook: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/CC-EST2023-SYASEX.pdf

    The script to download all 52 datasets individually and compile the data into
    a single aggregated .csv dataset is saved in our project folder as build_syasex.py
    https://drive.google.com/file/d/1D9Jh9shEhlZrEkVL-c4L6o4cKHf2ZU8p/view?usp=sharing

    The file created by the script is named: cc-est2023-syasex-all.csv
    and is saved in our DATASETS folder
    """
    # Google Drive file ID to access the specific population estimates dataset
    file_id = '12DzLf7Wvn-u1qF9Ez4rHxEEcKB9QOVOG'
    # Generate the download URL using the Google Drive file ID
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    # Read the CSV data into a pandas DataFrame
    df = pd.read_csv(file_url, dtype = {
        "SUMLEV": str,
        "STATE": str,
        "COUNTY": str,
    })
    df = df[df["YEAR"] == 5]
    df["FIPS"] = df["STATE"] + df["COUNTY"]
    # Select only the relevant columns from the dataset
    df["State"] = df["STNAME"].apply(states_to_abbreviations.get)
    df = df[["FIPS", "STATE", "State", "STNAME", "COUNTY", "CTYNAME", "AGE", "TOT_POP", "TOT_MALE", "TOT_FEMALE"]]
    df = df.reset_index()
    # Map full state names to their corresponding abbreviations using the 'states_to_abbreviations' dictionary
    return df

# Load the population estimates data and store it in a DataFrame, printing the head of the data for verification
population_estimates_county_df = load_population_estimates_county()
print(population_estimates_county_df)

          index   FIPS STATE State                    STNAME COUNTY  \
0           344  01001    01    AL                   Alabama    001   
1           345  01001    01    AL                   Alabama    001   
2           346  01001    01    AL                   Alabama    001   
3           347  01001    01    AL                   Alabama    001   
4           348  01001    01    AL                   Alabama    001   
...         ...    ...   ...   ...                       ...    ...   
277087  1385455  72153    72  None  Puerto Rico Commonwealth    153   
277088  1385456  72153    72  None  Puerto Rico Commonwealth    153   
277089  1385457  72153    72  None  Puerto Rico Commonwealth    153   
277090  1385458  72153    72  None  Puerto Rico Commonwealth    153   
277091  1385459  72153    72  None  Puerto Rico Commonwealth    153   

                CTYNAME  AGE  TOT_POP  TOT_MALE  TOT_FEMALE  
0        Autauga County    0      705       358         347  
1        Autauga County

In [ ]:
# Function to download, extract, and load U.S. county boundaries data using Census TIGER/Line Shapefiles.
def load_county_boundaries():
    """
    From the Census TIGER/Line Shapefiles
    Page: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html
    Datasets: https://www2.census.gov/geo/tiger/TIGER2024/COUNTY/
    Filename: tl_2024_us_county.zip
    url for direct download: https://www2.census.gov/geo/tiger/TIGER2024/COUNTY/tl_2024_us_county.zip
    """
    file_id = '13JfRr7bFxCt0WYW3qm5XH-jwyEMc-jOj'
    file_url = google_drive_file_download_url.substitute(file_id=file_id)

    # Need to use a different endpoint and add `confirm=t` query param to download "can't scan for viruses" files
    file_url = file_url.replace('uc?', 'download?') + '&confirm=t'
    print(file_url)

    extraction_path = os.path.join('content', 'tl_2024_us_county')
    response = requests.get(file_url)
    # It's necessary to extract the files into the (temporary) colab working directory because geopandas needs to see all the files at once
    with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zf:
        zf.extractall(extraction_path)

    # Find the (unique) filename for the main shapefile from the extracted files
    shp_fn = next(fn for fn in os.listdir(extraction_path) if fn.endswith('.shp'))
    # Combine the extraction_path and the shp_filename into a shp_filepath
    shp_fp = os.path.join(extraction_path, shp_fn)

    gdf = gpd.read_file(shp_fp).sort_values("GEOID")
    # Convert the state FIPS codes in the "STATEFP" column to state abbreviations using the dictionary `FIPS_to_abbreviations`
    # This adds a new column called "State" with the corresponding state abbreviations
    gdf["State"] = gdf["STATEFP"].apply(FIPS_to_abbreviations.get)
    return gdf

gdf_COUNTY = load_county_boundaries()
gdf_COUNTY.drop(columns='geometry').to_csv('content/counties_without_geometry.csv')
print(gdf_COUNTY.head())
Name_to_GEOID = gdf_COUNTY.set_index(["State", "NAME"]).GEOID
print(Name_to_GEOID)

https://drive.usercontent.google.com/download?id=13JfRr7bFxCt0WYW3qm5XH-jwyEMc-jOj&authuser=0&export=download&confirm=t
     STATEFP COUNTYFP  COUNTYNS  GEOID         GEOIDFQ     NAME  \
1323      01      001  00161526  01001  0500000US01001  Autauga   
1242      01      003  00161527  01003  0500000US01003  Baldwin   
1829      01      005  00161528  01005  0500000US01005  Barbour   
3016      01      007  00161529  01007  0500000US01007     Bibb   
853       01      009  00161530  01009  0500000US01009   Blount   

            NAMELSAD LSAD CLASSFP  MTFCC CSAFP CBSAFP METDIVFP FUNCSTAT  \
1323  Autauga County   06      H1  G4020   388  33860     None        A   
1242  Baldwin County   06      H1  G4020   380  19300     None        A   
1829  Barbour County   06      H1  G4020  None  21640     None        A   
3016     Bibb County   06      H1  G4020   142  13820     None        A   
853    Blount County   06      H1  G4020   142  13820     None        A   

           ALAND      AWAT

In [ ]:
# Function to download, extract, and load U.S. state boundaries data using Census TIGER/Line Shapefiles.
def load_state_boundaries():
    """
    From the Census TIGER/Line Shapefiles
    Page: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html
    Datasets: https://www2.census.gov/geo/tiger/TIGER2024/COUNTY/
    Filename: tl_2024_us_county.zip
    url for direct download: https://www2.census.gov/geo/tiger/TIGER2024/COUNTY/tl_2024_us_county.zip
    """
    file_id = '1vbfbBcl5ZH-JGFd9FrCV9x6arl2m-Rth'
    file_url = google_drive_file_download_url.substitute(file_id=file_id)

    extraction_path = os.path.join('content', 'tl_2024_us_state')
    response = requests.get(file_url)
    # It's necessary to extract the files into the (temporary) colab working directory because geopandas needs to see all the files at once
    with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zf:
        zf.extractall(extraction_path)

    # Find the (unique) filename for the main shapefile from the extracted files
    shp_fn = next(fn for fn in os.listdir(extraction_path) if fn.endswith('.shp'))
    # Combine the extraction_path and the shp_filename into a shp_filepath
    shp_fp = os.path.join(extraction_path, shp_fn)

    gdf = gpd.read_file(shp_fp)
    gdf["State"] = gdf["STATEFP"].apply(FIPS_to_abbreviations.get)
    return gdf

gdf_STATE = load_state_boundaries()
print(gdf_STATE.head())

  REGION DIVISION STATEFP   STATENS GEOID      GEOIDFQ STUSPS           NAME  \
0      3        5      54  01779805    54  0400000US54     WV  West Virginia   
1      3        5      12  00294478    12  0400000US12     FL        Florida   
2      2        3      17  01779784    17  0400000US17     IL       Illinois   
3      2        4      27  00662849    27  0400000US27     MN      Minnesota   
4      3        5      24  01714934    24  0400000US24     MD       Maryland   

  LSAD  MTFCC FUNCSTAT         ALAND       AWATER     INTPTLAT      INTPTLON  \
0   00  G4000        A   62266513826    488918898  +38.6472854  -080.6183274   
1   00  G4000        A  138965379385  45968913048  +28.3989775  -082.5143005   
2   00  G4000        A  143778206717   6216848695  +40.1028754  -089.1526108   
3   00  G4000        A  206244791203  18937236061  +46.3159573  -094.1996043   
4   00  G4000        A   25151223822   6979843236  +38.9466584  -076.6744939   

                                      

In [ ]:
# Function to download and load electric vehicle registration data by state from the U.S. Department of Energy.
def load_registrations_state_level():
    """
    Electric Vehicle Registrations by State by the Alternative Fuels Data Center (U.S. Department of Energy)
    The specific dataset is https://afdc.energy.gov/data/10962
    The list of datasets is at https://afdc.energy.gov/data/categories/maps-data-categories?per_page=150

    Filename:
    10962-ev-registration-counts-by-state_9-06-24.xlsx

    URL for direct download:
    https://afdc.energy.gov/files/u/data/data_source/10962/10962-ev-registration-counts-by-state_9-06-24.xlsx?12518e7893
    """
    file_id = "1glCUH55GRLn-WCp3QIEoToyEYpXwpsmi"
    file_url = google_drive_file_download_url.substitute(file_id=file_id)

    # Read the Excel file into a pandas DataFrame
    # - skiprows=2: Skips the first two rows to get to the data
    # - usecols=[1, 2]: Loads only the relevant columns (State name and registration count)
    df = pd.read_excel(file_url, skiprows=2, usecols=[1,2])
    # Rename the column 'State' to 'STATE_NAME' for consistency with other datasets
    df.rename(columns = {"State": "STATE_NAME"}, inplace=True)
    return df

registrations_state_df = load_registrations_state_level()
# Map state names to their abbreviations using the 'states_to_abbreviations' dictionary
registrations_state_df["State"] = registrations_state_df["STATE_NAME"].apply(states_to_abbreviations.get)
print(registrations_state_df.head())

   STATE_NAME  Registration Count State
0     Alabama               13047    AL
1      Alaska                2697    AK
2     Arizona               89798    AZ
3    Arkansas                7108    AR
4  California             1256646    CA


In [ ]:
def load_registrations_county_level():
    """
    Electric Vehicle Registrations by County in 18 states, by Atlas EV Hub
    https://www.atlasevhub.com/materials/state-ev-registration-data/

    The script to download all 18 datasets individually and compile the data into
    a single aggregated .csv dataset is saved in our project folder as build_atlasevhub.py
    https://drive.google.com/file/d/1rpRLxOLxBxKv_zI64h42_2WQfkYDQ0Cx/view?usp=drive_link

    The file created by the script is named: Atlas_EV_Hub_EV_Registrations.csv
    and is saved in our DATASETS folder
    """
    file_id = "15lj2RGEuGtN32pbPvbcM-5wUj_uNcqQ2"
    file_url = google_drive_file_download_url.substitute(file_id=file_id)

    df = pd.read_csv(file_url, dtype={
        "County GEOID": str,
        "ZIP Code": str,
        "County": str,})
    df["State"] = df["State"].apply(states_to_abbreviations.get)

    # hot-fixes:
    df['County'] = df['County'].str.replace(' County$', '', regex=True)
    df['County'] = df['County'].str.replace(' city$', '', regex=True)
    df.drop(df[df['County'].isin(['Other', 'Unknown'])].index, inplace=True)
    df.loc[(df['State'] == 'FL') & (df['County'] == 'Dade'), 'County'] = 'Miami-Dade'
    df.loc[df['County'] == 'Desoto', 'County'] = 'DeSoto'
    df.loc[(df['State'] == 'FL') & (df['County'] == 'Gadsen'), 'County'] = 'Gadsden'
    df["County GEOID"] = df.apply(
        lambda row: row["County GEOID"] if pd.isna(row["County"]) else Name_to_GEOID.loc[(row["State"], row["County"])].iloc[0],
        axis = 1 )
    df.drop(columns="County", inplace=True)

    df = df.merge(
        right = ZipCode_FIPS_df[["ZIP", "COUNTY", "TOT_RATIO"]],
        how="left",
        left_on="ZIP Code", right_on="ZIP",
    )
    # Eliminate the portions of ZIP codes that are in states outside of the registering agency:
    df = df[
        pd.notna(df["County GEOID"])
        | (df["State"] == df["COUNTY"].apply(gdf_COUNTY.set_index("GEOID").State.get))
    ]

    df['FIPS'] = df['County GEOID'].combine_first(df['COUNTY'])
    df["EV Registrations"] = df["EV Registrations"] * df["TOT_RATIO"].fillna(1.0)
    df.drop(columns=["County GEOID", "ZIP Code", "ZIP", "COUNTY", "TOT_RATIO"], inplace=True)

    # Before aggregating all registration years together:
    df = df.groupby(["Registration Year", "FIPS"])["EV Registrations"]\
        .sum().reset_index().sort_values(["FIPS", "Registration Year"])
    print(df)

    # After aggregating all registration years together:
    df = df.groupby("FIPS")["EV Registrations"]\
        .sum().sort_index().reset_index().set_index("FIPS")
    df["State"] = gdf_COUNTY.set_index("GEOID").State
    df["County Name"] = gdf_COUNTY.set_index("GEOID").NAME
    df["Actual Registrations in State"] = df["State"].apply(
        registrations_state_df.set_index("State")["Registration Count"].get)
    df.dropna(subset="Actual Registrations in State", inplace=True)
    df["Aggregated Registrations in State"] = df["State"].apply(
        df.groupby("State")["EV Registrations"].sum().get )
    print(df)
    df["EV Registrations"] = df["EV Registrations"] * (df["Actual Registrations in State"] / df["Aggregated Registrations in State"])
    df["EV Registrations"] = df["EV Registrations"].round().astype(int)
    df = df[["EV Registrations", "State", "County Name"]]
    print(df)

    return df

registrations_county_df = load_registrations_county_level()

<ipython-input-9-29e0689ee079>:30: PerformanceWarning: indexing past lexsort depth may impact performance.
  lambda row: row["County GEOID"] if pd.isna(row["County"]) else Name_to_GEOID.loc[(row["State"], row["County"])].iloc[0],


      Registration Year   FIPS  EV Registrations
76                 2010  06001         20.000000
181                2011  06001        362.000000
389                2012  06001       1168.000000
641                2013  06001       3622.000000
923                2014  06001       8304.000000
...                 ...    ...               ...
3602               2018  55141         46.167064
4626               2019  55141         22.150042
5793               2020  55141         29.791379
6924               2021  55141         37.588877
7991               2022  55141         47.474578

[9706 rows x 3 columns]
       EV Registrations State County Name  Actual Registrations in State  \
FIPS                                                                       
06001     192172.000000    CA     Alameda                        1256646   
06003         38.000000    CA      Alpine                        1256646   
06005        768.000000    CA      Amador                        1256646   
06007  

In [ ]:
# Function to download and load Electric Vehicle (EV) fuel station data from a Kaggle dataset.
def load_fuel_stations():
    """
    Electric Vehicle fuel stations 2023.05.18 from kaggle
    https://www.kaggle.com/datasets/torinsrose1/ev-fuel-stations-usa-2023-5-18/data

    Filename:
    ev-fuel-stations-usa-2023-5-18.zip

    URL for direct download:
    https://www.kaggle.com/api/v1/datasets/download/torinsrose1/ev-fuel-stations-usa-2023-5-18
    """
    file_id = "1w1Midox6QTxCM3-7GhweerVpgNX9iSER"
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    # Define the CSV filename within the downloaded ZIP archive
    csv_fn = "ev_fuel_stations_5.18.2023.csv"


    # Send an HTTP GET request to download the ZIP file from Google Drive
    response = requests.get(file_url)
    # Open the downloaded content as a ZIP file using 'io.BytesIO'
    with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zf:
        # Extract the CSV file from the ZIP archive and load it into a pandas DataFrame
        with zf.open(csv_fn) as f: df = pd.read_csv(f)
    return df

fuel_stations_df = load_fuel_stations()
geometry = gpd.points_from_xy(fuel_stations_df['Longitude'], fuel_stations_df['Latitude'])
fuel_stations_gdf = gpd.GeoDataFrame(fuel_stations_df, geometry=geometry, crs="EPSG:4269")
fuel_stations_gdf["FIPS County"] = gpd.sjoin(fuel_stations_gdf, gdf_COUNTY, how="left", predicate="intersects").GEOID

print(fuel_stations_df.columns)

<ipython-input-11-842a7995640b>:24: DtypeWarning: Columns (3,6,20,31,45,46,54,63,69) have mixed types. Specify dtype option on import or set low_memory=False.
  with zf.open(csv_fn) as f: df = pd.read_csv(f)


Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

In [ ]:
# Function to download and process data on the highest level of education attainment from the US Department of Agriculture.
def load_completing_college():
    """
    Highest level of education attainmeent from US Department of Agriculture
    Cannot be downloaded from a stable url (requires active session with cookies)

    https://data.ers.usda.gov/reports.aspx?ID=17829

    Filename:
    EducationReport.xlsx
    """
    file_id = "1uSDfddUmGbR7H13OhdswxxJKyC_3nWZv"
    file_url = google_drive_file_download_url.substitute(file_id=file_id)

    # Read the Excel file into a pandas DataFrame
    # - skiprows=2: Skips the first two rows which may contain metadata or headers
    # - header=[0,1]: Indicates that the dataset has multi-level column headers
    # - index_col=0: Uses the first column as the index (state names)
    df = pd.read_excel(file_url, skiprows=2, header=[0,1], index_col=0)
    # Flatten multi-level column headers by joining levels with an underscore
    df.columns = ['_'.join(map(str, col)).strip() for col in df.columns]
    # Remove the row corresponding to "United States" to focus on state-level data
    df = df[df.index != "United States"]
    # Map state names to their abbreviations using the 'states_to_abbreviations' dictionary
    # This renames the index to use state abbreviations instead of full state names
    df.rename(index=states_to_abbreviations.get, inplace=True)
    return df

# Function to extract the 'Total_2018-2022' column from the processed education data.
# This represents the percentage of the population completing higher education for each state.
def completing_college_by_state():
    college_df = load_completing_college()
    return college_df['Total_2018-2022']

college_Series = completing_college_by_state()
print(college_Series.head())


In [ ]:
#  Function to download and process personal mean income data by county from the U.S. Bureau of Economic Analysis (BEA).
def load_mean_income():
    """
    Personal (Mean) Income by County
    https://www.bea.gov/data/income-saving/personal-income-county-metro-and-other-areas

    Filename:
    lapi1123.xlsx

    URL for direct download:
    https://www.bea.gov/data/income-saving/personal-income-county-metro-and-other-areas
    """
    file_id = "1R_jxZgjuJy9Enf9eJOIyV0dTy8B5Vd01"
    file_url = google_drive_file_download_url.substitute(file_id=file_id)
    df = pd.read_excel(file_url, skiprows=2, header=[0,1])
    df.columns = ['_'.join(map(str, col)).strip() for col in df.columns]
    df.columns.values[0] = "NAME"
    df.dropna(how="all", inplace=True)
    df = df.iloc[0:-3]
    current_state = None
    for idx, row in df.iterrows():
        if row['Rank in state_2022'] == '--': current_state = row["NAME"]
        df.loc[idx, "State"] = current_state
    df["State"] = df["State"].apply(states_to_abbreviations.get)

    states_df  = df[(df["Rank in state_2022"] == '--') & (df["NAME"] != "United States")]
    states_df = states_df.reset_index().drop("index", axis=1)

    counties_df = df[df["Rank in state_2022"] != '--']
    FIPS_codes_df = load_FIPS_codes()

    # # hot-fixes:
    FIPS_codes_df.loc[FIPS_codes_df['STATE'] == 'VA', 'COUNTYNAME'] = FIPS_codes_df.loc[FIPS_codes_df['STATE'] == 'VA', 'COUNTYNAME'].str.replace( ' city$', '', regex=True)
    FIPS_codes_df['COUNTYNAME'] = FIPS_codes_df['COUNTYNAME'].str.replace(' County$', '', regex=True)
    FIPS_codes_df['COUNTYNAME'] = FIPS_codes_df['COUNTYNAME'].str.replace(' Parish$', '', regex=True)
    FIPS_codes_df['COUNTYNAME'] = FIPS_codes_df['COUNTYNAME'].str.replace(' City$', ' city', regex=True)

    counties_df['NAME'] = counties_df['NAME'].str.split(r' \+').str[0]
    counties_df['NAME'] = counties_df['NAME'].str.split(r',').str[0]
    counties_df['NAME'] = counties_df['NAME'].str.replace(' City$', ' city', regex=True)
    counties_df['NAME'] = counties_df['NAME'].str.replace('Augusta, Staunton + Waynesboro', 'Augusta')
    counties_df['NAME'] = counties_df['NAME'].str.replace('Fremont (includes Yellowstone National Park)', 'Fremont')
    counties_df['NAME'] = counties_df['NAME'].str.replace('Lagrange', 'LaGrange')
    counties_df = counties_df[~counties_df['NAME'].isin(['Independent cities:', 'Combination areas2:', 'St.'])]
    Name_to_FIPS = FIPS_codes_df.set_index(["STATE", "COUNTYNAME"]).FIPS
    counties_df["FIPS"] = counties_df.apply(
        lambda row: Name_to_FIPS.loc[(row["State"], row["NAME"])].iloc[0],
        axis = 1 )

    return states_df, counties_df

mean_income_df, mean_income_counties_df = load_mean_income()
mean_income_Series = mean_income_df.set_index("State")["Dollars_2022"]
mean_income_counties_Series = mean_income_counties_df.set_index("FIPS")["Dollars_2022"]
print(mean_income_Series.head())
print(mean_income_counties_Series.head())

# Build ABT (Analytic Base Table):

It is important to construct a well-organized and feature-rich ABT, to ensure we are laying the groundwork for more effective, efficient, and insightful data analysis, ultimately leading to better decision-making and predictive modeling outcomes.

More information on general ABTs: https://en.wikipedia.org/wiki/Analytical_base_table

In [ ]:
# Calculate the total population by state for the year 2023.
# Uses the 'load_population_estimates()' function to retrieve data.
# - Filters for both sexes (SEX = 0)
# - Filters for total population across all age groups (AGE = 999)
# - Sets the state abbreviation as the index
# - Returns a pandas Series containing total population estimates by state
def total_population_by_state():
    df = load_population_estimates()
    df = df[df["SEX"] == 0].drop("SEX", axis=1) # 0 means both sexes, see codebook
    df = df[df["AGE"] == 999].drop("AGE", axis=1)
    df = df.set_index("State")
    # Return the total population for each state
    return df["POPEST2023_CIV"]

# Calculate the median age for each state based on cumulative population estimates.
# - Filters for both sexes (SEX = 0)
# - Excludes the total population row (AGE = 999)
# - Sorts data by state and age for accurate cumulative summation
# - Calculates the median age using cumulative population interpolation
# - Returns a pandas Series with the median age by state
def median_age_by_state():
    df = load_population_estimates()
    df = df[df["SEX"] == 0].drop("SEX", axis=1) # 0 means both sexes, see codebook above
    df = df[df["AGE"] != 999] # Drops the row with 999 which indicates the total population
    df = df.sort_values(["State", "AGE"]) #  Sort data by state and age to ensure cumulative sum is calculated correctly
    # Calculate the median age for a single state using cumulative population.
    # - Creates a cumulative sum of the population by age
    # - Identifies the median age using interpolation
    def median_age(state):
        state["cumulative_POP"] = state["POPEST2023_CIV"].cumsum()
        median_person = state["POPEST2023_CIV"].sum() / 2
        state.reset_index(inplace=True) # Reset index to ensure it is numeric
        # Iterate through rows to find the median age by interpolation
        for i, row in state.iterrows():
            if row["cumulative_POP"] >= median_person:
                age1 = (0 if i == 0 else state.iloc[i-1]["AGE"])
                age2 = row["AGE"]
                pop1 = 0 if i == 0 else state.iloc[i-1]["cumulative_POP"]
                pop2 = row["cumulative_POP"]
                # Calculate interpolative distance to estimate the median age
                interpolative_distance = (median_person - pop1) / (pop2 - pop1)
                interpolated_age = interpolative_distance * (age2 - age1) + age1
                return interpolated_age
    # Group by state and apply the median age calculation
    return df.groupby('State').apply(median_age)

# Calculate the sex ratio (males to females) for each state.
# - Filters for total population across all ages (AGE = 999)
# - Groups data by state to calculate the ratio of males to females
# - Returns a pandas Series with the sex ratio by state
def sex_ratio_by_state():
    df = load_population_estimates()
    df = df[df["AGE"] == 999].drop("AGE", axis=1)
    # Calculate the ratio of males to females for a single state.
    # - SEX = 1 represents males & SEX = 2 represents females
    # - Returns the ratio of males to females
    def sex_ratio(state):
        males = state[df["SEX"] == 1]["POPEST2023_CIV"].iloc[0]
        females = state[df["SEX"] == 2]["POPEST2023_CIV"].iloc[0]
        return males / females
    # Group by state and apply the sex ratio calculation
    return np.log( df.groupby('State').apply(sex_ratio) )

# Load and calculate total population by state
population_Series = total_population_by_state()
# Load and calculate median age by state
median_age_Series = median_age_by_state()
# Load and calculate sex ratio by state
sex_ratio_Series  = sex_ratio_by_state()
# Load and assign integer of accessibility type by state
accessibility_Series  = load_fuel_stations()

In [ ]:
def total_population_by_county():
    df = population_estimates_county_df.copy()
    return df.groupby("FIPS").TOT_POP.sum()

def median_age_by_county():
    df = population_estimates_county_df.copy()
    df = df.sort_values(["FIPS", "AGE"]) #  Sort data by state and age to ensure cumulative sum is calculated correctly
    def median_age(county):
        county["cumulative_POP"] = county["TOT_POP"].cumsum()
        median_person = county["TOT_POP"].sum() / 2
        county.reset_index(inplace=True) # Reset index to ensure it is numeric
        # Iterate through rows to find the median age by interpolation
        for i, row in county.iterrows():
            if row["cumulative_POP"] >= median_person:
                age1 = (0 if i == 0 else county.iloc[i-1]["AGE"])
                age2 = row["AGE"]
                pop1 = 0 if i == 0 else county.iloc[i-1]["cumulative_POP"]
                pop2 = row["cumulative_POP"]
                # Calculate interpolative distance to estimate the median age
                interpolative_distance = (median_person - pop1) / (pop2 - pop1)
                interpolated_age = interpolative_distance * (age2 - age1) + age1
                return interpolated_age
    # Group by state and apply the median age calculatio9n
    return df.groupby('FIPS').apply(median_age)

def sex_ratio_by_county():
    df = population_estimates_county_df.copy()
    # Calculate the ratio of males to females for a single state.
    # - SEX = 1 represents males & SEX = 2 represents females
    # - Returns the ratio of males to females
    sum_df = df.groupby("FIPS")[["TOT_POP", "TOT_MALE", "TOT_FEMALE"]].sum()
    return np.log( sum_df["TOT_MALE"] / sum_df["TOT_FEMALE"] )

population_County_Series = total_population_by_county()
print(population_County_Series)
median_age_County_Series = median_age_by_county()
print(median_age_County_Series)
sex_ratio_County_Series = sex_ratio_by_county()
print(sex_ratio_County_Series)


In [ ]:
print("Computing county areas") # Prints a message indicating that the calculation of county areas is beginning
gdf_usa_albers_projection = gdf_COUNTY.to_crs('EPSG:5070')
gdf_usa_albers_projection["Area_km2"] = gdf_usa_albers_projection.geometry.area / 1e6
# Group the data by the "State" column and sum the areas of all counties within each state
# This step calculates the total area of each state by aggregating the areas of its counties.
state_areas = gdf_usa_albers_projection.groupby('State').Area_km2.sum()
print(state_areas.head())

In [ ]:
# Creating our own dataset:

# Create a DataFrame called "States_ABT" with initial columns for state names and EV registration counts
# This data comes from the "registrations_state_df" DataFrame
States_ABT = registrations_state_df[["State", "Registration Count"]]
# Group the fuel stations data by state to get the count of stations in each state
# The "size()" method counts the number of occurrences (fuel stations) per state
fuel_stations_by_state = fuel_stations_df.groupby("State").size()

States_ABT["Fuel Station Count"] = States_ABT["State"].apply(fuel_stations_by_state.get) # Add the count of fuel stations to the "States_ABT" DataFrame
States_ABT["Population"] = States_ABT["State"].apply(population_Series.get) # Add the total population for each state to the DataFrame
States_ABT["Median Age"] = States_ABT["State"].apply(median_age_Series.get) # Add the median age for each state
States_ABT["log Sex Ratio"] = States_ABT["State"].apply(sex_ratio_Series.get) # Add the sex ratio (males to females) for each state
States_ABT["Completing College"] = States_ABT["State"].apply(college_Series.get) # Add the percentage of people completing college for each state
States_ABT["Mean Income"] = States_ABT["State"].apply(mean_income_Series.get) # Add the mean income for each state
States_ABT["Area_km2"] = States_ABT["State"].apply(state_areas.get) # Add the total area in square kilometers for each state
States_ABT["Pop_per_km2"] = States_ABT["Population"] / States_ABT["Area_km2"] # Calculate the population density (people per square kilometer) for each state
States_ABT["Fuel Stations per capita"] = States_ABT["Fuel Station Count"] / States_ABT["Population"] # Calculate the number of fuel stations per capita for each state
States_ABT["EV Registrations per capita"] = States_ABT["Registration Count"] / States_ABT["Population"] # Calculate the number of EV registrations per capita for each state

States_ABT = States_ABT.iloc[:-1]
States_ABT = States_ABT.set_index('State')
print(States_ABT.head())

In [ ]:
EV = States_ABT
EV.to_csv('EV.csv')
from google.colab import files
files.download('EV.csv')

In [ ]:
County_ABT = registrations_county_df[["EV Registrations", "State"]]
County_ABT["Fuel Station Count"] = County_ABT.index.map(fuel_stations_gdf.groupby("FIPS County").size().get).fillna(0).astype(int)
County_ABT["Population"] = County_ABT.index.map(population_County_Series.get)
County_ABT["Median Age"] = County_ABT.index.map(median_age_County_Series.get)
County_ABT["log Sex Ratio"] = County_ABT.index.map(sex_ratio_County_Series.get)
County_ABT["Completing College"] = County_ABT["State"].apply(college_Series.get) # Add the percentage of people completing college for each state

County_ABT["Mean Income"] = County_ABT.index.map(mean_income_counties_Series.get) # Add the mean income for each county
County_ABT = County_ABT[pd.to_numeric(County_ABT['Mean Income'], errors='coerce').notna()]
County_ABT["Mean Income"] = County_ABT["Mean Income"].astype(float)

County_ABT["Area_km2"] = County_ABT.index.map(gdf_usa_albers_projection.set_index("GEOID")["Area_km2"].get)
County_ABT["Pop_per_km2"] = County_ABT["Population"] / County_ABT["Area_km2"] # Calculate the population density (people per square kilometer) for each county
County_ABT["Fuel Stations per capita"] = County_ABT["Fuel Station Count"] / County_ABT["Population"] # Calculate the number of fuel stations per capita for each county
County_ABT["EV Registrations per capita"] = County_ABT["EV Registrations"] / County_ABT["Population"] # Calculate the number of EV registrations per capita for each county

County_ABT.drop(columns="State", inplace=True)
County_ABT.dropna(inplace=True)
print(County_ABT)

# Descriptive Statistics & Other



In [ ]:
States_ABT.describe()

In [ ]:
print( States_ABT.info() ) # .info() method in pandas provides a summary of the DataFrame - structure & contents
print( County_ABT.info() )

In [ ]:
print( States_ABT.describe() ) # .describe() provides count, mean, std, min, 25%, 50%, 75%, & max
print( County_ABT.describe() ) # .describe() provides count, mean, std, min, 25%, 50%, 75%, & max

In [ ]:
print( States_ABT["Registration Count"].sum() )
print( County_ABT["EV Registrations"].sum() )

# Data Analysis:

In the context of our project, which focuses on analyzing electric vehicle (EV) adoption and infrastructure across different states, data analysis plays a key role in identifying the factors that drive EV registrations. By leveraging demographic, economic, and infrastructure data, we aim to uncover patterns that can inform strategic decisions, such as where to prioritize the expansion of EV charging stations to maximize their impact.

To make our findings more accessible, we present multiple visualizations that showcase critical insights derived from our analysis. For example, we use bar charts to compare EV registrations and charging stations per capita across states, scatter plots to explore the relationship between charging infrastructure and EV adoption, and maps to visualize geographical patterns in EV accessibility. These visualizations not only help us communicate our analysis effectively but also provide actionable insights for stakeholders looking to enhance EV infrastructure and policy planning.


In [ ]:
# Generates a scatter plot with a regression line to visualize the relationship
# between fuel stations per capita and EV registrations per capita in the States_ABT DataFrame
sns.regplot(x="Fuel Stations per capita", y="EV Registrations per capita", data=States_ABT)
for idx, row in States_ABT.iterrows():
    plt.text(
        row["Fuel Stations per capita"], row["EV Registrations per capita"], idx,
        fontsize=8, ha='right', va='bottom'
    )

plt.title('Relationship Between Fuel Stations and EV Registrations',
          fontsize=16, fontweight='bold', fontname='STIXGeneral')
plt.xlabel('Fuel Stations Per Capita', fontsize=14, fontname='STIXGeneral')
plt.ylabel('EV Registrations Per Capita', fontsize=14, fontname='STIXGeneral')
plt.xscale('log')
plt.yscale('log')
plt.show()

# The result allows us to see outliers - like California.
# The line has an overall positive slope, indicating a positive correlation, suggesting
# that states with more fuel stations per capita tend to have higher EV adoption.

In [ ]:
sns.regplot(x="Fuel Stations per capita", y="EV Registrations per capita", data=County_ABT)

plt.title('Relationship Between Fuel Stations and EV Registrations',
          fontsize=16, fontweight='bold', fontname='STIXGeneral')
plt.xlabel('Fuel Stations Per Capita', fontsize=14, fontname='STIXGeneral')
plt.ylabel('EV Registrations Per Capita', fontsize=14, fontname='STIXGeneral')
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
# This code creates a map visualization (geographic boundary map) to show the locations of fuel stations across
# the United States and plots the fuel station locations on top of it using latitude and longitude coordinates.
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the state boundaries from the GeoDataFrame
gdf_STATE.boundary.plot(ax=ax, linewidth=1, edgecolor='black')

# Plot latitude and longitude points
public_stations  = fuel_stations_df[fuel_stations_df["Groups With Access Code"].str.contains("Public")]
private_stations = fuel_stations_df[fuel_stations_df["Groups With Access Code"].str.contains("Private")]
plt.scatter(public_stations['Longitude'], public_stations['Latitude'], color='red', marker='o', s=1, label='Public EV fuel stations', alpha=.2)
plt.scatter(private_stations['Longitude'], private_stations['Latitude'], color='blue', marker='o', s=2, label='Private EV fuel stations', alpha=1)

ax.set_xlim(-180, -60)
ax.set_ylim(15, 75)

# Adding titles and labels
plt.title("EV Fuel Stations Over U.S. States Wireframe", fontsize=20, fontweight='bold', fontname='STIXGeneral')
plt.xlabel("Longitude", fontsize=15, fontname='STIXGeneral')
plt.ylabel("Latitude", fontsize=15, fontname='STIXGeneral')

plt.show()

# The result of this code allows us to visually assess geographic coverage and identify areas with high or low densities of fuel stations.

In [ ]:
# This code creates a BAR PLOT to visualize the Electric Vehicle (EV) registrations per capita for each state
# using the data from the States_ABT DataFrame. The visualization highlights how EV adoption varies across different states.

evR = States_ABT.reset_index()[['State', 'EV Registrations per capita']]
evR.plot(kind='bar', stacked=True, figsize=(15, 8), color='pink', edgecolor='black') # Makes it pretty
plt.title('Electric Vehicle Registrations by State', size=25, fontweight='bold', fontname='STIXGeneral')
plt.xlabel('State', fontsize=20, labelpad=20, fontname='STIXGeneral')
plt.ylabel('EV Registration Per Capita', fontsize=20, labelpad=20, fontname='STIXGeneral')
plt.xticks(ticks=np.arange(len(evR['State'])), labels=evR['State'], fontsize=10, fontname='STIXGeneral')
plt.yticks(fontsize=15, fontname='STIXGeneral')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend(['EV Registrations Per Capita'], prop={'family': 'STIXGeneral', 'size': 12})

# The result identifies states with with high or low EV adoption rates

In [ ]:
# This code generates a bar plot to visualize the number of electric vehicle (EV) fuel stations per
# capita across various states. It uses data from the States_ABT DataFrame to show how fuel
# station accessibility varies from state to state, which can be critical for understanding infrastructure readiness for EV adoption.

evR = States_ABT.reset_index()[['State', 'Fuel Stations per capita']]
evR.plot(kind='bar', stacked=True, figsize=(15, 8), color='pink', edgecolor='black') # Makes it pretty)
plt.title('Electric Vehicle Stations by State', size=25, fontweight='bold', fontname='STIXGeneral')
plt.xlabel('State', fontsize=20, labelpad=20, fontname='STIXGeneral')
plt.ylabel('Fuel Stations Per Capita', fontsize=20, labelpad=20, fontname='STIXGeneral')
plt.xticks(ticks=np.arange(len(evR['State'])), labels=evR['State'], fontsize=10, fontname='STIXGeneral')
plt.yticks(fontsize=15, fontname='STIXGeneral')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend(['Fuel Stations Per Capita'], prop={'family': 'STIXGeneral', 'size': 12})

# The result identifies states with high or low accessibility to EV charging stations

In [ ]:
# This code creates a HORIZONTAL BOX PLOT that visualizes the distribution of the Fuel Station Count per state.

plt.figure(figsize=(10, 6))
plt.boxplot(States_ABT['Fuel Station Count'].dropna(),
            patch_artist=True,
            vert=False,
            boxprops=dict(facecolor='pink'),
            medianprops=dict(color='green'))

plt.yticks([1], ['Fuel Station Count'], fontname='STIXGeneral', fontsize=14)
plt.xlabel('Fuel Stations Per Capita', fontname='STIXGeneral', fontsize=14)
plt.title('Fuel Station Count', fontsize=18, fontweight='bold', fontname='STIXGeneral')
plt.grid(True, axis='x')
plt.show()

# The result gives a visual summary of how evenly or unevenly fuel stations are distributed among the states.

In [ ]:
# This code generates a HISTOGRAM to visualize the distribution of fuel stations per capita across different states
# using the data from the States_ABT DataFrame

evH = States_ABT['Fuel Stations per capita']
plt.hist(evH, bins=30, color='pink', edgecolor='black') # Makes the graph prettier
plt.title('Fuel Stations Per Capita', fontsize=14, fontname='STIXGeneral', fontweight='bold')
plt.xlabel('Value', fontsize=14, labelpad=12, fontname='STIXGeneral')
plt.ylabel('Number of States', fontsize=14, labelpad=12, fontname='STIXGeneral')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xticks(fontsize=12, fontname='STIXGeneral')
plt.yticks(fontsize=12, fontname='STIXGeneral')

plt.axvline(evH.mean(), color='purple', linestyle='dashed', linewidth=1, label='Mean') # Added the Mean function
plt.axvline(evH.median(), color='green', linestyle='dashed', linewidth=1, label='Median') # Added the Median function
plt.legend()

# The result shows the frequency distribution of fuel stations per capita across all states.
# This shows whether most states have high or low access to fuel stations relative to their population.
# If there are bars that are far away from the rest of the distribution, they indicate states
# that are outliers in terms of fuel station availability.

In [ ]:
evH = States_ABT['Mean Income']
plt.hist(evH, bins=30, color='dodgerblue', edgecolor='black') # Makes the graph prettier
plt.title('Mean Income Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Income ($)', fontsize=14, labelpad=12)
plt.ylabel('Number of States', fontsize=14, labelpad=12)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.axvline(evH.mean(), color='purple', linestyle='dashed', linewidth=1, label='Mean') # Added the Mean function
plt.axvline(evH.median(), color='green', linestyle='dashed', linewidth=1, label='Median') # Added the Median function
plt.legend()

In [ ]:
# This code generates a HISTOGRAM that visualizes the distribution of electric
# vehicle (EV) registrations per capita across different states.
evH = States_ABT['EV Registrations per capita']
plt.hist(evH, bins=30, color='pink', edgecolor='black') # Makes the graph prettier
plt.title('EV Registrations Per Capita', fontname='STIXGeneral', fontsize=14, fontweight='bold')
plt.xlabel('Value', fontsize=14, labelpad=12, fontname='STIXGeneral')
plt.ylabel('Number of States', fontsize=14, labelpad=12, fontname='STIXGeneral')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xticks(fontsize=12, fontname='STIXGeneral')
plt.yticks(fontsize=12, fontname='STIXGeneral')

plt.axvline(evH.mean(), color='purple', linestyle='dashed', linewidth=1, label='Mean') # Added the Mean function
plt.axvline(evH.median(), color='green', linestyle='dashed', linewidth=1, label='Median') # Added the Median function
plt.legend()

# The result provides a snapshot of how EV adoption varies across states

In [ ]:
plt.hist(States_ABT, bins=30, edgecolor='black')
plt.title('EV Registrations Per Capita', fontsize=14, fontname='STIXGeneral', fontweight='bold')
plt.xlabel('Value', fontsize=14, labelpad=12, fontname='STIXGeneral')
plt.ylabel('Number of States', fontsize=14, labelpad=12, fontname='STIXGeneral')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Build Models

In [ ]:
def make_best_random_forest_model(X_train, X_test, y_train, y_test):    # Define the model
    rf = RandomForestRegressor()

    # Set the parameter grid for hyperparameter tuning
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True, False]
    }

    # Use GridSearchCV to optimize hyperparameters with cross-validation
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    # Best parameters and model performance
    best_params = grid_search.best_params_
    best_randomized_forest_model = grid_search.best_estimator_
    best_score = -grid_search.best_score_  # Mean squared error

    print("Best Parameters:", best_params)
    print("Best Cross-Validation MSE:", best_score)

    # Evaluate on a test set (if available)
    best_randomized_forest_model.fit(X_train, y_train)
    y_pred = best_randomized_forest_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    print("Test MSE:", test_mse)
    return best_randomized_forest_model

In [ ]:
def make_best_ridge_model(X_train, X_test, y_train, y_test):
    ridge = Ridge()

    # Set the parameter grid for hyperparameter tuning (e.g., regularization strength)
    param_grid = {
        'alpha': [0.01, 0.1, 1, 10, 100]
    }

    # Use GridSearchCV to optimize hyperparameters with cross-validation
    grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    # Best parameters and model performance
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    best_score = -grid_search.best_score_  # Mean squared error

    print("Best Parameters:", best_params)
    print("Best Cross-Validation MSE:", best_score)

    # Evaluate on a test set (if available)
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    print(f"MSE of best model: {mean_squared_error(y_test, y_pred)}")
    print(f"r^2 of best model: {r2_score(y_test, y_pred)}")

    return best_model


In [ ]:
def build_state_models(ridge=False, forest=False):
    # Load data into X (features) and y (target variable)
    target = 'Registration Count'
    X = States_ABT.drop(columns=[target, 'EV Registrations per capita'])
    y = States_ABT[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if ridge:
        best_ridge_model = make_best_ridge_model(X_train, X_test, y_train, y_test)
        best_idx, best_diff_EV_registrations = None, 0
        for idx, row in X.iterrows():
            hypothetical_row = row.copy()
            hypothetical_row['Fuel Station Count'] += 1
            y_pred = best_ridge_model.predict([row])
            hypothetical_y_pred = best_ridge_model.predict([hypothetical_row])

            diff_EV_registrations = float((hypothetical_y_pred - y_pred))
            if diff_EV_registrations > best_diff_EV_registrations:
                best_idx = idx
                best_diff_EV_registrations = diff_EV_registrations

        print(best_idx, best_diff_EV_registrations)

    if forest:
        best_random_forest_model = make_best_random_forest_model(X_train, X_test, y_train, y_test)
        best_idx, best_diff_EV_registrations = None, 0
        for idx, row in X.iterrows():
            hypothetical_row = row.copy()
            hypothetical_row['Fuel Station Count'] += 1
            y_pred = best_random_forest_model.predict([row])
            hypothetical_y_pred = best_random_forest_model.predict([hypothetical_row])

            diff_EV_registrations = float((hypothetical_y_pred - y_pred))
            if diff_EV_registrations > best_diff_EV_registrations:
                best_idx = idx
                best_diff_EV_registrations = diff_EV_registrations
        print(best_idx, best_diff_EV_registrations)

build_state_models(ridge=True)

In [ ]:
def build_county_models(ridge=False, forest=False):
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    # Load data into X (features) and y (target variable)
    target = 'EV Registrations'
    X = County_ABT.drop(columns=[target, 'EV Registrations per capita'])
    y = County_ABT[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if ridge:
        best_ridge_model = make_best_ridge_model(X_train, X_test, y_train, y_test)
        best_idx, best_diff_EV_registrations = None, 0
        for idx, row in X.iterrows():
            hypothetical_row = row.copy()
            hypothetical_row['Fuel Station Count'] += 1
            y_pred = best_ridge_model.predict([row])
            hypothetical_y_pred = best_ridge_model.predict([hypothetical_row])

            diff_EV_registrations = float((hypothetical_y_pred - y_pred))
            if diff_EV_registrations > best_diff_EV_registrations:
                best_idx = idx
                best_diff_EV_registrations = diff_EV_registrations

        print(best_idx, best_diff_EV_registrations)

    if forest:
        best_random_forest_model = make_best_random_forest_model(X_train, X_test, y_train, y_test)
        best_idx, best_diff_EV_registrations = None, 0
        for idx, row in X.iterrows():
            hypothetical_row = row.copy()
            hypothetical_row['Fuel Station Count'] += 1
            y_pred = best_random_forest_model.predict([row])
            hypothetical_y_pred = best_random_forest_model.predict([hypothetical_row])

            diff_EV_registrations = float((hypothetical_y_pred - y_pred))
            if diff_EV_registrations > best_diff_EV_registrations:
                best_idx = idx
                best_diff_EV_registrations = diff_EV_registrations
        print(best_idx, best_diff_EV_registrations)

In [ ]:
build_county_models(ridge=True, forest=False)

In [ ]:
build_county_models(ridge=False, forest=True)